# Prep

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

In [ ]:
from bson.objectid import ObjectId

def show_doc(db, collection, id):
    '''
    Finds a document by 'id' and prints contents to the console
    
    Parameters:
    --------
    db         : database name
    collection : mongodb collection
    id         : mongodb document id
    
    Returns:
    --------
    Prints first 100 symbols of each document's key to console
    '''
    
    doc = db[collection].find_one({'_id':ObjectId(id)})
    for k in doc:
        print(f"{k} : {str(doc[k])}")

# Some comands to keep dbs clean

In [ ]:
# deletes all 'meta' fields from all docs
# htmlCol.update({}, {$unset: {meta:1}}, false, true); # mongo shell comand
htmlCol.update({}, {'$unset': {'meta':1}}, multi=True) # pymongo way

In [ ]:
# leaves only unique documents by 'url' field

htmlCol.create_index(
    "url",
    unique=True
)

In [ ]:
# pymongo 'find' returns cursor that allows iterating through results
# calling first object [0] allows accessing the dictionary with results
# the ['html'] is the key in the dictionary
html = htmlCol.find({'url':'http://www.msnbc.com/velshi-ruhle/watch/jeff-sessions-is-justifying-harsh-immigration-policy-with-the-bible-1256689731629'},\
            projection={'html':True, '_id':False})[0]['html']

In [ ]:
# find documents NOT containing a 'tag': regex expression
import re
tag = re.compile('dek___3AQpw.')
docs = htmlCol.find({"html" : {'$not': tag}})
for d in docs[:20]: print(d['url'])

# Multithreading

In [ ]:
from multiprocessing import Process

# use multiprocessing to extract features
def func():
    DB_NAME = 'scrape'
    db = pm.MongoClient(host='localhost', port=27017, maxPoolSize=500)[DB_NAME]

    for collection in ['left','right']: docs_parser(db[collection])

proc = Process(target=func)
proc.start()

# Helpers

In [ ]:
# For debugging turn on logging to console
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

# dependencies
import pymongo as pm

from resources.newspaper import newspaper
from resources.newspaper.newspaper.source import Source
from resources.newspaper.newspaper.article import Article

import bs4 as bs
from urllib.parse import urljoin
from dateutil.parser import parse as date_parser
from time import sleep
import random
import pytz

import requests

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn, maxPoolSize=200)

# define db 
# DB_NAME = 'scrape'
DB_NAME = 'FINALP'
db = client[DB_NAME]

In [ ]:
# powered by NewsAPI.org
SOURCES = {
    'left'  : [
        'https://newrepublic.com',
        'https://www.motherjones.com'
# 3. Slate
# 4. The Intercept
# 5. Daily Beast
# 6. The Atlantic
# 7. Washington Post
# 8. Politico
# 9. The Guardian
# 10. BBC
    ],
    'right' : [
        'https://www.breitbart.com'
# 2. Fox News
# 3. New York Post
# 4. The American Conservative
# 5. Washington Times
# 6. Daily Wire
# 7. The Fiscal Times
# 8. The Hill
# 9. The Daily Caller
# 10. Reason
    ]
}

In [ ]:
def saveToDB(db, collection, url, html, meta={}):
    """
    Saves a document to mongoDB, making sure there are no duplicates by 
    'url' value
    
    Parameters:
    --------
    db, collection  : mongo db connection
    url, html, meta : values to store
    
    Returns:
    --------
    Saved document
    """
    collection = db[collection]
    collection.update_one(
        {'url' : url},
        {
            '$set':
                {'url' : url,
                 'html' : html,
                 'meta' : meta
                }
        }
        ,
        upsert=True
    )
    log.debug(f'Saved to DB')

In [ ]:
import datetime
def scrape(url, db, collection):
    '''
    Scrapes an article from the 'url' up to the 'latest_date'
    
    Parameters:
    --------
    url         : main news website url
    date        : YYYY-MM-DD
    db          : database name
    collection  : mongodb collection
    
    Returns:
    --------
    Article's html and features stored to db, 
    Article's publish date
    
    '''
    log.debug(f"Exctracting features from {url}")
    try:
        article = Article(url)
        article.download()
        # the below method may only extract a snippet... 
        # check the database for results of text extraction
        # and apply additional processing if needed after 
        # article has been stored in the DB
        # see code below Newrepublic for example
        article.parse()
    except Exception as e:
        log.critical(f'Data not saved: {e}')
        return datetime.datetime.now()
    
    saveToDB(db, collection, article.url, article.html, meta={
        'date'    :article.publish_date,
        'title'   :article.title,
        'text'    :article.text,
        'authors' :article.authors
    })
    
    return article.publish_date

# Newrepublic

In [ ]:
# name of collection for this media
collection = 'newRep'
source = 'https://newrepublic.com/latest'
page   = 91

earliest_date = date_parser('2017-01-01')

while True:
    log.debug(f'PROCESSING PAGE: {page}')
    s = Source(source+'?page='+str(page))
    s.download()

    soup = bs.BeautifulSoup(s.html,'lxml')

    # line below needs to be updated per news source
    # to include the specific tags for article text 
    # defined differently for each site
    for section in soup.findAll('article'):
        url = urljoin(s.url, section.a['href'])
        log.debug(f'Processing url: {url}')
        
        article_date = scrape(url, db, collection)

    if article_date < earliest_date:
        log.debug(f'Reached earliest date requested: {article_date}')
        break
    page += 1

In [ ]:
# update text field to include more data
for doc in db[collection].find():
    soup = bs.BeautifulSoup(doc['html'],'lxml')
    text = ''
    for div in soup.findAll('div',{"class": "content-body"}):
        text += div.text
    if len(doc['meta']['text']) < len(text):
        db[collection].update_one(
            {'url' : doc['url']},
            {
                '$set':
                    {
                     'meta.text' : text
                    }
            }
            ,
            upsert=True
        )

In [ ]:
url = 'https://newrepublic.com/article/139550/legacy-altamont'
doc = db[collection].find_one({'url':url})
doc['meta']['text']

# The Atlantic

In [ ]:
utc=pytz.UTC

# name of collection for this media
collection = 'theAtlantic'
source = 'https://www.theatlantic.com/latest/'

# start 'page' at '1' but if you run across an error
# efficient way is to update this page to the same number
# where you experienced the error AFTER you correct the error in the code
# then rerun this cell
page   = 175

earliest_date = utc.localize(date_parser('2017-01-01'))

while True:
    log.debug(f'\n\n PROCESSING PAGE: {source+"?page="+str(page)}\n\n\
              ====================================\n\n')
    s = Source(source+'?page='+str(page))
    page += 1
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')

    # line below needs to be updated per news source
    # to include the specific tags for article text 
    # defined differently for each site
    for section in soup.findAll('li', {"class":"article"}):
        url = urljoin(s.url, section.a['href'])
        log.debug(f'Processing url: {url}')
        
        try:
            article_date = scrape(url, db, collection)
        except Exception:
            article_date = earliest_date + 1 #to make sure scraping continues
        
        # the Atlantic blocks right away after few quick downloads
        # so it requires sleeping, testing showed 1 to 5 seconds is enough
#         sleep(random.uniform(1,5))

    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f'Exception: {e}')
        continue
    

# Politico

In [ ]:
# setup
# use your newsapi key
# update collection, date and source
collection = 'Politoco'
page = 1
earliest_date = '2017-01-01'
latest_date = '2017-09-13'
source = 'politico'
pageSize = 100
apiKey = 'fd1386b678fd4524b2aa84e3bee1f8f7'

params = {
    'apiKey' : apiKey,
    'pageSize' : pageSize,
    'page' : page,
    'from' : earliest_date,
    'to'   : latest_date,
    'sources': source
}

# base url
api_url = 'https://newsapi.org/v2/everything?'

# get first request to obtain total news count
r = requests.get(api_url, params=params)
totalPages = r.json()['totalResults']//100
log.debug(f'Total pages ({pageSize} per page) for {source} results from {earliest_date} to {latest_date} is {totalPages}')

# scrape news
for p in range(page,totalPages+1):
    log.debug(f'\n\n PROCESSING PAGE: {source+"?page="+str(page)}\n\n\
              ====================================\n\n')
    
    params = {
        'apiKey' : apiKey,
        'pageSize' : pageSize,
        'page' : page,
        'from' : earliest_date,
        'to'   : latest_date,
        'sources': source
    }
    
    r = requests.get(api_url, params=params)

    page += 1
    
    if r.json()['articles']:
        for a in r.json()['articles']:
            try:
                log.debug(f"Processing url: {a['url']}")
                article_date = scrape(a['url'], db, collection)
            except Exception:
                article_date = earliest_date + 1
    else:
        log.debug(f"Response doesn't have articles. Response: {r.json()}")
        

In [ ]:
r.json()['totalResults']

# Washington Times

In [ ]:
# setup
# use your newsapi key
# update collection, date and source
collection = 'WashTimes'

In [ ]:
page = 1
earliest_date = '2017-01-01'
latest_date = '2017-12-31'
source = 'the-washington-times'
pageSize = 100
apiKey = 'fd1386b678fd4524b2aa84e3bee1f8f7'

params = {
    'apiKey'   : apiKey,
    'pageSize' : pageSize,
    'page'     : page,
    'from'     : earliest_date,
    'to'       : latest_date,
    'sources'  : source
}

# base url
api_url = 'https://newsapi.org/v2/everything?'

# get first request to obtain total news count
r = requests.get(api_url, params=params)
totalPages = r.json()['totalResults']//100

log.debug(f'TOTAL PAGES FOR {source}: {totalPages}')

# scrape news
for p in range(page,totalPages):
    log.debug(f'\n\n PROCESSING PAGE: {source+"?page="+str(page)}\n\n\
              ====================================\n\n')
    
    params = {
        'apiKey'   : apiKey,
        'pageSize' : pageSize,
        'page'     : page,
        'from'     : earliest_date,
        'to'       : latest_date,
        'sources'  : source
    }
    
    r = requests.get(api_url, params=params)

    page += 1
    
    for a in r.json()['articles']:
        try:
            log.debug(f"Processing url: {a['url']}")
            if db[collection].find({'url':a['url']}):
                log.debug(f'URL exists in DB. Skipping.')
            else:
                article_date = scrape(a['url'], db, collection)
        except Exception:
            article_date = earliest_date + 1

In [ ]:
r.json()

# Slate

In [ ]:
collection = 'slate'
source = 'https://slate.com/news-and-politics/'
page   = 118

utc=pytz.UTC
earliest_date = utc.localize(date_parser('2017-01-01'))

while True:
    log.debug(f'\n\n PROCESSING PAGE: {source+str(page)}\n\n\
              ====================================\n\n')
    s = Source(source+str(page))
    page += 1
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')

    # line below needs to be updated per news source
    # to include the specific tags for article text 
    # defined differently for each site
    for link in soup.find('div', {"class":"topic-stories-list"}).findChildren("a" , recursive=False):
        url = link['href']
        log.debug(f'Processing url: {url}')
        
        try:
            article_date = scrape(url, db, collection)
        except Exception:
            article_date = earliest_date + 1 #to make sure scraping continues
        
        # the Atlantic blocks right away after few quick downloads
        # so it requires sleeping, testing showed 1 to 5 seconds is enough
#         sleep(random.uniform(1,5))

    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f'Exception: {e}')
        continue

# The Intercept

In [ ]:
collection = 'theintercept'
source = 'https://theintercept.com'

# for infinite scroll page
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import sys

import unittest, time, re

logging.getLogger('selenium').setLevel(logging.WARNING)

In [ ]:
class Sel(unittest.TestCase):
    def setUp(self):
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(30)
        self.base_url = source
        self.verificationErrors = []
        self.accept_next_alert = True
    def getPage(self):
        driver = self.driver
        delay = 2
        driver.get(self.base_url)
        html_source = driver.page_source
        self.html = html_source.encode('utf-8')
        return self.html
    def scrollDown(self):
        driver = self.driver
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.html = driver.page_source.encode('utf-8')
        return driver.page_source 
    def shutdown(self):
        driver = self.driver
        driver.quit()

data = Sel()

In [ ]:
data.setUp()
data.getPage()
page = 1

utc=pytz.UTC
earliest_date = utc.localize(date_parser('2017-01-01'))

scraped_urls = []

while True:
    log.debug(f'NEXT SCROLL (#{page})')
    page += 1
    
    html = data.scrollDown()
    soup = bs.BeautifulSoup(html,'lxml')

    for link in soup.find('div', {"id":"recently"}).findAll('a'):
        url = urljoin(source, link['href'])
        if url and url not in scraped_urls:
            scraped_urls.append(url)
            log.debug(f'Processing url: {url}')
            article_date = scrape(url, db, collection)

    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f"Something is wrong: {e}")
data.shutdown()

In [ ]:
data.shutdown()

# The daily beast

In [12]:
collection = 'thedailybeast'
source = 'https://www.thedailybeast.com/sitemap'
earliest_date = date_parser('2017-01-01')
s = Source(source)
s.download()
soup = bs.BeautifulSoup(s.html,'lxml')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /sitemap HTTP/1.1" 200 None


In [21]:
months = ['https://www.thedailybeast.com/sitemap/2018/1/article',
 'https://www.thedailybeast.com/sitemap/2018/2/article',
 'https://www.thedailybeast.com/sitemap/2018/3/article',
 'https://www.thedailybeast.com/sitemap/2018/4/article',
 'https://www.thedailybeast.com/sitemap/2018/5/article',
 'https://www.thedailybeast.com/sitemap/2018/6/article',
 'https://www.thedailybeast.com/sitemap/2018/7/article',
 'https://www.thedailybeast.com/sitemap/2017/1/article',
 'https://www.thedailybeast.com/sitemap/2017/2/article',
 'https://www.thedailybeast.com/sitemap/2017/3/article',
 'https://www.thedailybeast.com/sitemap/2017/4/article',
 'https://www.thedailybeast.com/sitemap/2017/5/article',
 'https://www.thedailybeast.com/sitemap/2017/6/article',
 'https://www.thedailybeast.com/sitemap/2017/7/article',
 'https://www.thedailybeast.com/sitemap/2017/8/article',
 'https://www.thedailybeast.com/sitemap/2017/9/article',
 'https://www.thedailybeast.com/sitemap/2017/10/article',
 'https://www.thedailybeast.com/sitemap/2017/11/article',
 'https://www.thedailybeast.com/sitemap/2017/12/article']

In [ ]:
for month in months:
    log.debug(f"\n=======PROCESSING PAGE: {month}\n")
    s = Source(month)
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')
    for a in soup.find('div',{'class':'SitemapMonthPage__articles-wrapper'}).findAll('a'):
        url = urljoin(source, a['href'])
        log.debug(f'Processing url: {url}')
        
        article_date = scrape(url, db, collection) 

DEBUG:__main__:
=======PROCESSING PAGE: https://www.thedailybeast.com/sitemap/2018/1/article

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /sitemap/2018/1/article HTTP/1.1" 200 None
DEBUG:__main__:Processing url: https://www.thedailybeast.com/roger-stone-says-he-visited-the-ecuadorian-embassy-in-london-where-julian-assange-is-in-hiding
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/roger-stone-says-he-visited-the-ecuadorian-embassy-in-london-where-julian-assange-is-in-hiding
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /roger-stone-says-he-visited-the-ecuadorian-embassy-in-london-where-julian-assange-is-in-hiding HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/a-top-russian-spys-visit-to-dc

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/trumps-male-pattern-baldness-drugdemystified
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/trumps-male-pattern-baldness-drugdemystified
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /trumps-male-pattern-baldness-drugdemystified HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/steve-wynn-proves-rnc-only-cares-about-sexual-assault-when-it-involves-dems
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/steve-wynn-proves-rnc-only-cares-about-sexual-assault-when-it-involves-dems
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /steve-wynn-proves-rnc-only-cares-about-sexual-assault-when-it-invo

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /jamaica-bans-anti-gay-american-preacher-steven-anderson HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/devin-nunes-wont-say-if-he-worked-with-white-house-on-anti-fbi-memo
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/devin-nunes-wont-say-if-he-worked-with-white-house-on-anti-fbi-memo
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /devin-nunes-wont-say-if-he-worked-with-white-house-on-anti-fbi-memo HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/trans-soldier-going-to-state-of-the-union-tells-trump-you-can-count-on-people-like-me
DEBUG:__main__:Exctracting features from https://www.thedai

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/2018-state-of-the-union-how-to-watch-live-stream
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/2018-state-of-the-union-how-to-watch-live-stream
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /2018-state-of-the-union-how-to-watch-live-stream HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/judd-apatow-fires-back-at-diane-keaton-over-woody-allen-defense
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/judd-apatow-fires-back-at-diane-keaton-over-woody-allen-defense
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /judd-apatow-fires-back-at-diane-keaton-over-woody-allen-defense HTTP/1.1" 200 

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/stephen-colbert-mocks-nikki-haleys-grammys-outrage
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/stephen-colbert-mocks-nikki-haleys-grammys-outrage
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /stephen-colbert-mocks-nikki-haleys-grammys-outrage HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/trump-administration-baffles-and-enrages-lawmakers-with-latest-punt-on-russia-sanctions
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/trump-administration-baffles-and-enrages-lawmakers-with-latest-punt-on-russia-sanctions
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /trump-administration-baf

...

DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /the-nigerian-prince-scam-gets-the-1-million-movie-treatment HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/msnbc-host-joy-reid-doubles-down-on-claim-that-hackers-fabricated-her-homophobic-blog-posts
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/msnbc-host-joy-reid-doubles-down-on-claim-that-hackers-fabricated-her-homophobic-blog-posts
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /msnbc-host-joy-reid-doubles-down-on-claim-that-hackers-fabricated-her-homophobic-blog-posts HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/stephen-colbert-roasts-trumps-failed-attempt-to-hold-melanias-hand-she-was-not-having-it
DEBUG:__main__:Exctracting features from https://www.thedailybeast

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /alek-minassian-elliot-rodger-toronto-van-murder-mens-rights-incel-mra HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/this-wireless-charger-can-power-an-iphone-and-an-apple-watch-simultaneously
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/this-wireless-charger-can-power-an-iphone-and-an-apple-watch-simultaneously
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /this-wireless-charger-can-power-an-iphone-and-an-apple-watch-simultaneously HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/brighten-your-workspace-with-this-levitating-moon-lamp
DEBUG:__main__:Exctracting features from https://www

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/ex-epa-officials-legal-advocacy-outfit-embroiled-in-vicious-leadership-struggle
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/ex-epa-officials-legal-advocacy-outfit-embroiled-in-vicious-leadership-struggle
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /ex-epa-officials-legal-advocacy-outfit-embroiled-in-vicious-leadership-struggle HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/kate-and-william-had-their-third-child-why-and-what-were-we-watching
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/kate-and-william-had-their-third-child-why-and-what-were-we-watching
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.the

DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/south-beach-wine-and-food-festival-wraps-up-another-successful-year
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /south-beach-wine-and-food-festival-wraps-up-another-successful-year HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/sarah-huckabee-sanders-democrats-voting-against-pompeo-dont-love-america
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/sarah-huckabee-sanders-democrats-voting-against-pompeo-dont-love-america
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /sarah-huckabee-sanders-democrats-voting-against-pompeo-dont-love-america HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://w

DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/how-the-army-recruits-straight-out-of-prisons
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /how-the-army-recruits-straight-out-of-prisons HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.thedailybeast.com/democrats-are-primed-to-win-big-reclaim-the-house-and-save-our-democracy-heres-how-they-could-blow-it
DEBUG:__main__:Exctracting features from https://www.thedailybeast.com/democrats-are-primed-to-win-big-reclaim-the-house-and-save-our-democracy-heres-how-they-could-blow-it
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.thedailybeast.com
DEBUG:urllib3.connectionpool:https://www.thedailybeast.com:443 "GET /democrats-are-primed-to-win-big-reclaim-the-house-and-save-our-democracy-heres-how-they-could-blow-it HTTP/1.1" 200 None
DEBUG:__main__:Saved to 